<a href="https://colab.research.google.com/github/sunilypatil/starter-slapp-app/blob/master/NVRs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
import random
from PIL import Image, ImageDraw, ImageFont
import os
import math
from PIL import ImageFont
from PIL import ImageOps
from math import sin, cos, radians

font = ImageFont.load_default()

In [50]:
from PIL import ImageOps

QUESTION_SIZE = 150

def generate_random_shape():
    shapes = ['circle', 'semicircle', 'oval', 'triangle', 'square', 'rectangle', 
              'parallelogram', 'rhombus', 'trapezoid', 'kite', 'pentagon', 'hexagon', 
              'heptagon', 'octagon']
    shape = random.choice(shapes)
    return shape


def draw_shape(draw, shape, bounding_box, fill='black'):
    x1, y1, x2, y2 = bounding_box
    center = ((x1 + x2) // 2, (y1 + y2) // 2)
    radius = min(abs(x2 - x1), abs(y2 - y1)) // 2

    if shape == 'circle':
        draw.ellipse(bounding_box, fill=fill)
    elif shape == 'semicircle':
        draw.pieslice(bounding_box, start=0, end=180, fill=fill)
    elif shape == 'oval':
        draw.ellipse(bounding_box, fill=fill)
    elif shape == 'triangle':
        triangle_points = [(x1, y2), (x2, y2), center]
        draw.polygon(triangle_points, fill=fill)
    elif shape == 'square':
        draw.rectangle(bounding_box, fill=fill)
    elif shape == 'rectangle':
        draw.rectangle(bounding_box, fill=fill)
    elif shape == 'parallelogram':
        points = [(x1, y2), (x1 + (x2-x1)//3, y1), (x2, y1), (x2 - (x2-x1)//3, y2)]
        draw.polygon(points, fill=fill)
    elif shape == 'rhombus':
        points = [(center[0], y1), (x2, center[1]), (center[0], y2), (x1, center[1])]
        draw.polygon(points, fill=fill)
    elif shape == 'trapezoid':
      # For trapezoid
        points = [(x1 + (x2-x1)//4, y2), (x1 + 3*(x2-x1)//4, y2), (x2, y1), (x1, y1)]
        draw.polygon(points, fill=fill)
    elif shape == 'kite':
        points = [(center[0], y1), (x2, center[1]), (center[0], y2), (x1, center[1])]
        draw.polygon(points, fill=fill)
    else:
        # For regular polygons
        num_sides = {'pentagon': 5, 'hexagon': 6, 'heptagon': 7, 'octagon': 8}[shape]
        points = []
        for i in range(num_sides):
            angle = radians(float(i) / num_sides * 360.0)
            x = center[0] + radius * cos(angle)
            y = center[1] + radius * sin(angle)
            points.append((x, y))
        draw.polygon(points, fill=fill)

def draw_square(draw, pos, size):
    x, y = pos
    draw.rectangle([x - size // 2, y - size // 2, x + size // 2, y + size // 2], outline="black", width=2)

def draw_circle(draw, pos, size):
    x, y = pos
    draw.ellipse([x - size // 2, y - size // 2, x + size // 2, y + size // 2], outline="black", width=2)

def draw_triangle(draw, pos, size):
    x, y = pos
    points = [(x, y - size // 2), (x + size // 2, y + size // 2), (x - size // 2, y + size // 2), (x, y - size // 2)]
    for i in range(3):
        draw.line([points[i], points[i + 1]], fill="black", width=2)


def draw_shape_with_points(draw, points, size):
    for i, point in enumerate(points):
        # Draw arcs between points
        angle = math.atan2(point[1] - points[i - 1][1], point[0] - points[i - 1][0])
        start_angle = math.degrees(angle)
        end_angle = math.degrees(angle + math.pi / len(points))

        bounding_box = [point[0] - size / 2, point[1] - size / 2, point[0] + size / 2, point[1] + size / 2]
        draw.arc(bounding_box, start_angle, end_angle, fill=(0, 0, 0), width=2)

    # Connect the last point to the first point
    draw.line([points[-1], points[0]], fill=(0, 0, 0), width=2)


def generate_complex_shape(draw, center, size, num_sides, angle_offset, arc_direction):
    # Create a complex shape using the provided attributes
    points = []
    for i in range(num_sides):
        angle = math.radians(i * 360 / num_sides + angle_offset)
        x = center[0] + size * math.cos(angle)
        y = center[1] + size * math.sin(angle)
        points.append((x, y))

        # Draw arcs between points
        start_angle = math.degrees(angle)
        end_angle = math.degrees(angle + math.pi / num_sides)
        if arc_direction:
            start_angle, end_angle = end_angle, start_angle

        bounding_box = [points[-1][0] - size / 2, points[-1][1] - size / 2, points[-1][0] + size / 2, points[-1][1] + size / 2]
        draw.arc(bounding_box, start_angle, end_angle, fill=(0, 0, 0), width=2)

    # Connect the last point to the first point
    draw.line([points[-1], points[0]], fill=(0, 0, 0), width=2)

    return points



def draw_arrow(draw, center, direction):
    if direction == "clockwise":
        points = [(center[0], center[1] - 10), (center[0] + 10, center[1]), (center[0], center[1] + 10)]
    else:
        points = [(center[0], center[1] - 10), (center[0] - 10, center[1]), (center[0], center[1] + 10)]
    draw.polygon(points, fill=(0, 0, 0))

def create_rotation_options(question, direction, degrees):
    answer_options = []
    rotations = [0, 90, 180, 270]
    random.shuffle(rotations)

    for rotation in rotations:
        option = question.copy()

        if direction == "clockwise":
            rotated_option = option.rotate(-rotation, Image.NEAREST, expand=1)
        else:
            rotated_option = option.rotate(rotation, Image.NEAREST, expand=1)

        rotated_option = rotated_option.resize((QUESTION_SIZE, QUESTION_SIZE), Image.LANCZOS)
        answer_options.append(rotated_option)

    if direction == "clockwise":
        correct_answer = rotations.index(degrees) + 1
    else:
        correct_answer = rotations.index(360 - degrees) + 1

    return answer_options, correct_answer


def generate_rotation_question():
    question = Image.new('RGBA', (QUESTION_SIZE, QUESTION_SIZE), 'white')
    draw = ImageDraw.Draw(question)

    center = (QUESTION_SIZE // 2, QUESTION_SIZE // 2)
    size = 50
    
    num_sides = random.randint(3, 6)
    angle_offset = random.uniform(0, 360)
    arc_direction = random.choice(['cw', 'ccw'])

    generate_complex_shape(draw, center, size, num_sides, angle_offset, arc_direction)

    # Choose the rotation direction
    direction = random.choice(["clockwise", "counterclockwise"])
    degrees = random.choice([90, 180, 270])

    # Create answer options
    answer_options, correct_answer = create_rotation_options(question, direction, degrees)

    question_text = f"What picture do you get when you rotate the shape by {degrees} degrees {direction}?"
    explanation = f"The correct answer is option {correct_answer} because the shape is rotated by {degrees} degrees {direction}."

    return question, answer_options, correct_answer, question_text, explanation


def generate_reflection_question():
    # Create an image with a complex shape
    img = Image.new('RGBA', (100, 100), (255, 255, 255, 255))
    draw = ImageDraw.Draw(img)
    center = (50, 50)
    size = 50

    generate_complex_shape(draw, center, size)

    # Reflect the shape along the vertical axis (correct answer)
    correct_img = ImageOps.mirror(img)

    # Generate distractor images
    # Reflect the shape along the horizontal axis
    distractor_img_horizontal = ImageOps.flip(img)

    # Reflect the shape along the diagonal axis (top-left to bottom-right)
    distractor_img_diagonal1 = img.rotate(270, resample=Image.BICUBIC, fillcolor=(255, 255, 255, 255))
    distractor_img_diagonal1 = ImageOps.flip(distractor_img_diagonal1)

    # Reflect the shape along the diagonal axis (top-right to bottom-left)
    distractor_img_diagonal2 = img.rotate(270, resample=Image.BICUBIC, fillcolor=(255, 255, 255, 255))
    distractor_img_diagonal2 = ImageOps.mirror(distractor_img_diagonal2)

    # Randomize the order of answer options
    answer_options = [correct_img, distractor_img_horizontal, distractor_img_diagonal1, distractor_img_diagonal2]
    random.shuffle(answer_options)
    correct_answer = answer_options.index(correct_img) + 1  # Answer options are 1-indexed

    question_text = f"What picture do you get when you reflect the shape across the {axis} axis?"
    explanation = f"The correct answer is option {correct_answer} because the shape is reflected across the {axis} axis."

    return img, answer_options, correct_answer, question_text, explanation


def generate_similar_complex_shape(draw, center, size, shape_attributes):
    similar_shape_attributes = shape_attributes.copy()

    # Make small changes to shape attributes for the distractor
    angle_variation = random.uniform(-10, 10)
    arc_length_variation = random.uniform(-0.1, 0.1)
    position_variation = (random.uniform(-5, 5), random.uniform(-5, 5))

    new_points = []
    for i, point in enumerate(shape_attributes['points']):
        angle = math.atan2(point[1] - center[1], point[0] - center[0])
        angle += math.radians(angle_variation)
        distance = math.hypot(point[0] - center[0], point[1] - center[1])
        distance += size * arc_length_variation

        x = center[0] + distance * math.cos(angle) + position_variation[0]
        y = center[1] + distance * math.sin(angle) + position_variation[1]
        new_points.append((x, y))

    draw_shape_with_points(draw, new_points, size)


def generate_random_shape_attributes():
    shape_attributes = {
        'num_sides': random.randint(3, 7),
        'angle_offset': random.randint(0, 360),
        'arc_direction': random.choice([True, False]),
    }
    return shape_attributes

def generate_shape_matching_question():
    # Create an image with a complex shape (correct answer)
    img = Image.new('RGBA', (100, 100), (255, 255, 255, 255))
    draw = ImageDraw.Draw(img)
    center = (50, 50)
    size = 50

    shape_attributes = generate_random_shape_attributes()
    points = generate_complex_shape(draw, center, size, **shape_attributes)

    correct_img = img

    # Generate distractor images
    distractor_imgs = []
    shape_attributes['points'] = points
    for _ in range(3):
        distractor_img = Image.new('RGBA', (100, 100), (255, 255, 255, 255))
        draw = ImageDraw.Draw(distractor_img)
        generate_similar_complex_shape(draw, center, size, shape_attributes)
        distractor_imgs.append(distractor_img)

    # Randomize the order of answer options
    answer_options = [correct_img] + distractor_imgs
    random.shuffle(answer_options)
    correct_answer = answer_options.index(correct_img) + 1  # Answer options are 1-indexed

    question_text = "Which of the following images matches the shape in the question?"

    return img, answer_options, correct_answer, question_text




def generate_progression_question():
    # Define a list of shapes to choose from
    shapes = [draw_square, draw_circle, draw_triangle]

    # Choose a random shape
    shape = random.choice(shapes)

    # Choose a random starting number of shapes and progression step
    start_num = random.randint(1, 4)
    step = random.randint(1, 3)

    # Create a sequence of images with increasing number of shapes
    sequence = []
    for i in range(1, 6):
        img = Image.new('RGBA', (100, 100), (255, 255, 255, 255))
        draw = ImageDraw.Draw(img)
        num_shapes = start_num + step * (i - 1)

        for _ in range(num_shapes):
            pos = (random.randint(10, 90), random.randint(10, 90))
            size = random.randint(10, 30)
            shape(draw, pos, size)

        sequence.append(img)

    # Choose the correct next image in the sequence
    correct_img = sequence[-1]

    # Generate distractor images
    distractor_img1 = Image.new('RGBA', (100, 100), (255, 255, 255, 255))
    distractor_img2 = Image.new('RGBA', (100, 100), (255, 255, 255, 255))
    distractor_img3 = Image.new('RGBA', (100, 100), (255, 255, 255, 255))

    # Draw distractor images with different numbers of shapes
    distractor_images = [distractor_img1, distractor_img2, distractor_img3]
    for i, img in enumerate(distractor_images):
        draw = ImageDraw.Draw(img)
        num_shapes = start_num + step * 4 + i + 1

        for _ in range(num_shapes):
            pos = (random.randint(10, 90), random.randint(10, 90))
            size = random.randint(10, 30)
            shape(draw, pos, size)

    # Randomize the order of answer options
    answer_options = [correct_img, distractor_img1, distractor_img2, distractor_img3]
    random.shuffle(answer_options)
    correct_answer = answer_options.index(correct_img) + 1  # Answer options are 1-indexed

    question_text = "What is the next image in the progression?"
    explanation = f"The correct answer is option {correct_answer} because the sequence follows a pattern of adding {step} shapes in each step."

      # Remove the correct image from the sequence
    sequence.pop()

    return sequence, answer_options, correct_answer, question_text, explanation

  
    #return sequence, answer_options, correct_answer, question_text

NUM_SIDES = [3, 4, 5, 6, 7, 8]
ANGLE_OFFSETS = [0, 45, 90, 135, 180, 225, 270, 315]
ARC_DIRECTIONS = ['clockwise', 'counterclockwise']

def draw_complex_pattern(draw, center, radius, num_elements):
    for _ in range(num_elements):
        # Choose a random position for the shape
        x = random.randint(radius, QUESTION_SIZE - radius)
        y = random.randint(radius, QUESTION_SIZE - radius)

        # Choose a random number of sides for the shape
        num_sides = random.choice([3, 4, 5, 6])

        # Choose a random direction for the arc
        arc_direction = random.choice([-1, 1])

        # Draw the shape
        generate_complex_shape(draw, (x, y), radius, num_sides, 0, arc_direction)

def add_border(image, border_width=2, border_color=(0, 0, 0)):
    width, height = image.size
    bordered_image = Image.new('RGBA', (width + 2 * border_width, height + 2 * border_width), border_color)
    bordered_image.paste(image, (border_width, border_width))
    return bordered_image

def generate_pattern_completion_question():
    # Create a new image with space for two patterns and a question mark
    question = Image.new('RGBA', (3 * QUESTION_SIZE, QUESTION_SIZE), 'white')
    pattern1 = Image.new('RGBA', (QUESTION_SIZE, QUESTION_SIZE), 'white')
    pattern2 = Image.new('RGBA', (QUESTION_SIZE, QUESTION_SIZE), 'white')
    
    # Draw the first pattern with a random number of shapes
    num_elements1 = random.randint(1, 5)
    draw_pattern1 = ImageDraw.Draw(pattern1)
    draw_complex_pattern(draw_pattern1, (QUESTION_SIZE // 2, QUESTION_SIZE // 2), 50, num_elements1)
    
    # Draw the second pattern with a different number of shapes
    num_elements2 = random.randint(1, 5)
    while num_elements2 == num_elements1:
        num_elements2 = random.randint(1, 5)
    draw_pattern2 = ImageDraw.Draw(pattern2)
    draw_complex_pattern(draw_pattern2, (QUESTION_SIZE // 2, QUESTION_SIZE // 2), 50, num_elements2)
    
    # Paste the patterns and the question mark onto the question image
    question.paste(add_border(pattern1), (0, 0))
    question.paste(add_border(pattern2), (QUESTION_SIZE, 0))
    
    # Draw the question mark
    draw = ImageDraw.Draw(question)
    #font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 40)
    draw.text((2 * QUESTION_SIZE + QUESTION_SIZE//2, QUESTION_SIZE//2), "?", fill="black", font=font, anchor="mm")
    
    # The correct answer is the difference in the number of shapes
    correct_answer = num_elements2 - num_elements1
    explanation = f"The pattern follows the increase of {correct_answer} shape(s) from the first image to the second. So, the next pattern should have {num_elements2 + correct_answer} shapes."
    
    question_text = "What is the next shape in the pattern?"
    
    return question, [pattern1, pattern2], correct_answer, question_text, explanation



QUESTION_TYPES = {
    'rotation': generate_rotation_question,
    'reflection': generate_reflection_question,
    'shape_matching': generate_shape_matching_question,
    'progression': generate_progression_question,
    'pattern_completion': generate_pattern_completion_question,
}

In [51]:
def generate_question_set(topic, num_questions):
    question_set = []
    for _ in range(num_questions):
        question, answer_options, correct_answer, question_text, explanation = QUESTION_TYPES[topic]()
        question_set.append({
            'question': question,
            'answer_options': answer_options,
            'correct_answer': correct_answer,
            'question_text': question_text,
            'explanation': explanation,
        })
    return question_set




In [52]:
def save_images(question_set, output_folder):

    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)

    os.makedirs(output_folder)

    with open(os.path.join(output_folder, 'explanations.txt'), 'w') as f:
        for idx, question in enumerate(question_set, start=1):
            if isinstance(question['question'], list):
                for img_idx, img in enumerate(question['question'], start=1):
                    img.save(os.path.join(output_folder, f'question_{idx}_part_{img_idx}.png'))
            else:
                question['question'].save(os.path.join(output_folder, f'question_{idx}.png'))

            for option_idx, option in enumerate(question['answer_options'], start=1):
                option.save(os.path.join(output_folder, f'question_{idx}_option_{option_idx}.png'))
            
            f.write(f"Question {idx}:\n")
            f.write(question['explanation'] + '\n\n')




In [53]:
import shutil
from google.colab import files

def create_zip_and_download(question_set, output_folder='output', zip_name='questions.zip'):
    # Save images to output folder
    save_images(question_set, output_folder)

   # Save question data as a text file
    with open(os.path.join(output_folder, 'question_data.txt'), 'w') as f:
        for idx, question in enumerate(question_set, start=1):
            f.write(f"Question {idx}: {question['question_text']}\n")
            f.write(f"Correct Answer: Option {question['correct_answer']}\n")
            f.write("\n")

    # Create a zip file containing the output folder
    shutil.make_archive(zip_name[:-4], 'zip', output_folder)

    # Download the zip file
    files.download(zip_name)


In [54]:
if __name__ == '__main__':
    topic = input("Enter the topic: ").lower()
    num_questions = int(input("Enter the number of questions: "))
    
    if topic not in QUESTION_TYPES:
        print("Invalid topic. Available topics are:", ', '.join(QUESTION_TYPES.keys()))
        exit()
        
    question_set = generate_question_set(topic, num_questions)
    output_folder = "question_set"  # You can change this to any folder name you prefer
    save_images(question_set, output_folder)
    print("Question set generated successfully!")
    #question_set = generate_question_set(topic, num_questions)
    create_zip_and_download(question_set)



Enter the topic: rotation
Enter the number of questions: 20
Question set generated successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>